# 변수 선택

In [1]:
import pandas as pd

df = pd.read_csv("c:/workspace3/data/customer.csv")
df.columns

Index(['Int'l Plan', 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls',
       'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins',
       'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn'],
      dtype='object')

In [2]:
cols=["Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls','Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins','Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge','CustServ Calls']

X=df[cols]
y=df['Churn']
y.value_counts()

Churn
0    2850
1     483
Name: count, dtype: int64

In [3]:
#언더샘플링

from imblearn.under_sampling import RandomUnderSampler

X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X, y)
X_samp = pd.DataFrame(data=X_sample,columns=cols )
y_samp = pd.DataFrame(data=y_sample,columns=['Churn'])

df_samp=pd.concat([X_samp,y_samp],axis=1)
df_samp['Churn'].value_counts()

Churn
0    483
1    483
Name: count, dtype: int64

In [4]:
X=df_samp[cols]
y=df_samp['Churn']

In [5]:
#시간이 오래 걸림

from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# 선형모형만 가능함(rbf 커널에는 coef가 제공되지 않음)
# SVC의 경우 변수 선택이 잘 되지 않을 수 있으므로 DecisionTree도 함께 테스트

model = SVC(kernel="linear")

#model = DecisionTreeClassifier(random_state=0).fit(X, y)

# step=1 각 단계에서 제거할 변수의 개수

rfecv = RFECV(estimator=model, step=2, cv=StratifiedKFold(5),
#               분류기          
              scoring='accuracy')

rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=SVC(kernel='linear'), scoring='accuracy', step=2)

In [7]:
import numpy as np

print(rfecv.cv_results_['mean_test_score']) #교차검증으로 계산한 변수들의 중요도
print(rfecv.n_features_) #교차검증을 통해 선택된 변수 개수
print(rfecv.support_) #변수 선택 여부
print(rfecv.ranking_) #변수 선택 랭킹

[0.6097217  0.6097217  0.69463704 0.72252016 0.73599167 0.76914695
 0.75982586 0.75982586 0.7639656 ]
10
[ True  True  True  True False False False False  True False False  True
  True  True  True  True]
[1 1 1 1 3 2 2 4 1 3 4 1 1 1 1 1]


In [8]:
#선택된 변수들
print(X.columns[rfecv.support_])

#선택되지 않은 변수들
print(X.columns.difference(X.columns[rfecv.support_]))

Index(['Int'l Plan', 'VMail Plan', 'VMail Message', 'Day Mins', 'Eve Charge',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls'],
      dtype='object')
Index(['Day Calls', 'Day Charge', 'Eve Calls', 'Eve Mins', 'Night Calls',
       'Night Mins'],
      dtype='object')


In [9]:
df2=pd.DataFrame(list(zip(X.columns[rfecv.support_],rfecv.cv_results_['mean_test_score'])),
                columns=['변수명','평균점수'])
df2

,변수명,평균점수
0,Int'l Plan,0.609722
1,VMail Plan,0.609722
2,VMail Message,0.694637
3,Day Mins,0.722520
4,Eve Charge,0.735992
5,Night Charge,0.769147
6,Intl Mins,0.759826
7,Intl Calls,0.759826
8,Intl Charge,0.763966


In [10]:
df2.sort_values(by='평균점수', ascending=False)

,변수명,평균점수
5,Night Charge,0.769147
8,Intl Charge,0.763966
6,Intl Mins,0.759826
7,Intl Calls,0.759826
4,Eve Charge,0.735992
3,Day Mins,0.722520
2,VMail Message,0.694637
0,Int'l Plan,0.609722
1,VMail Plan,0.609722
